# Informative path planning 

* This is a framework for creating models for informative path planning. 
* A visualization and animation tool
* An interactive tool to move the robots
* The data model for the information source
* Robot strategies


In [ ]:
import itertools
import numpy as np
import ast
import math
import random
from functools import partial
import matplotlib.pyplot as plt


from ipywidgets import interact, Button, IntText, GridBox, Layout, VBox, HBox, Box, HTML, Output, Label, FloatSlider, Image
from IPython.display import display

from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, column, layout
from bokeh.models import CategoricalColorMapper, LinearColorMapper, ColumnDataSource, Text
from bokeh.plotting import figure, output_file
from bokeh.colors import RGB

from Environment import Environment, PollutionModelEnvironment, EpidemicSpreadEnvironment
from InformationModel import InformationModel, ScalarFieldInformationModel_stored_observation
from Robot import Robot
from Policy import GoToLocationPolicy, FollowPathPolicy, RandomWaypointPolicy

import logging
# logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.WARNING)

# World
Implements a world class which integrates an environment with one or more observables and several robots.

In [ ]:
class World:
    def __init__(self, env, im):
        self.env = env
        self.im = im
        self.width, self.height = env.width, env.height
        self.time = 0
        self.robots = []
                
    def add_robot(self, robot):
        self.robots.append(robot)
        
    def enact_policy(self, delta_t = 1.0):
        """For all the agents in the environment, it calls 
        their policies to allow for the scheduling of the actions."""
        for robot in self.robots:
            robot.enact_policy(delta_t)
        
    def proceed(self, delta_t = 1.0):
        """Implements the passing of time"""
        for robot in self.robots:
            robot.proceed(delta_t)
        self.env.proceed(delta_t)
        self.im.proceed(delta_t)
        self.time = self.time + delta_t
        

# Visualization and control
This class creates a visualization technique for the system and controls for the robot.

In [ ]:
class IPPUI:
    def __init__(self, world):
        self.world = world
        
        
    def create_robot_control_panel(self, robot):
        """Creates a panel that allows for the control and status visualization panel for a given robot."""

        # Label - the name of the robot
        label = Label(value=robot.name)
        
        # button box for the manual entering of a robot action
        btn_north  = Button(description='North', layout=Layout(width='auto', grid_area='btn_north'))
        btn_west = Button(description='West', layout=Layout(width='auto', grid_area='btn_west'))
        btn_east = Button(description='East', layout=Layout(width='auto', grid_area='btn_east'))
        btn_south  = Button(description='South', layout=Layout(width='auto', grid_area='btn_south'))

        for button in [btn_north, btn_west, btn_east, btn_south]:
            button.on_click(partial(self.on_button_clicked_robot, robot))

        arrows = GridBox(children=[btn_north, btn_west, btn_east, btn_south], layout=Layout(
                    width='100%',
                    grid_area=robot.name, 
                    grid_template_rows='auto',
                    grid_template_columns='auto',
                    grid_template_areas='''
                    ". btn_north btn_north ."
                    "btn_west btn_west btn_east btn_east"
                    ". btn_south btn_south ."
                    '''
                ))
        # FIXME: create a description of the current status of the robot
        status = HTML(value=robot.toHTML())
        self.dict_robot_status[robot.name] = status
        
        panel = VBox([label, arrows, status], layout=Layout(border="solid 2px"))
        return panel
    
    def create_world_control_panel(self):
        """Creates a panel that allows for the control and status visualization for all robots and the 
        advance of the status of the world."""
        robot_panels = []
        grid_template_areas=""
        self.dict_robot_status = {} # mapping name to status HTML widget
        for robot in self.world.robots:
            robot_panels.append(self.create_robot_control_panel(robot))
        robot_control_panels = HBox(robot_panels)
        
        
        # apply policy button
        btn_apply_policy = Button(description = "Apply policies", layout = Layout(width='20%'))
        btn_apply_policy.on_click(partial(self.on_button_clicked_apply_policy))
        # apply actions button
        btn_apply_actions = Button(description = "Apply actions", layout = Layout(width='20%'))
        btn_apply_actions.on_click(partial(self.on_button_clicked_apply_actions))
        btn_proceed_all = Button(description = "Policy + Action", layout = Layout(width='20%'))
        btn_proceed_all.on_click(partial(self.on_button_clicked_proceed_all))        
        self.widget_timestep = FloatSlider(value = 1.0, min = 0.1, max = 3.0, step = 0.1)
        ## FIXME update this widget for the world time - maybe this should be a HTML instead???
        self.html_world_time = HTML(value="<b>T</b>=0")
        world_panel = HBox([btn_apply_policy, btn_apply_actions, btn_proceed_all, 
                            self.widget_timestep, self.html_world_time])
        #btn_proceed.on_click(partial(self.on_button_clicked_proceed))
        full_control_panel = VBox([robot_control_panels, world_panel])
        display(full_control_panel)

    def on_button_clicked_robot(self, robot, b):
        print(f"on_button_clicked_robot {robot.name} button {b}")
        robot.add_action(b.description)
        self.update_visualization()
        
    def on_button_clicked_proceed_all(self, b):
        """Action for the button Policy+Action"""
        self.world.enact_policy(self.widget_timestep.value)
        self.world.proceed(self.widget_timestep.value)
        self.update_visualization()

    def on_button_clicked_apply_policy(self, b):
        """Action for the button Apply policy"""
        self.world.enact_policy(self.widget_timestep.value)
        self.update_visualization()

    def on_button_clicked_apply_actions(self, b):
        """Action for the button Apply actions"""
        self.world.proceed(self.widget_timestep.value)
        self.update_visualization()
        
    def create_env_robot_visualization(self):
        """Creates the map visualization using bokeh"""
        output_notebook() # this shows that the output will be the notebook
        line_width = 10
        line_color = '#B8AD9A'  # the same for plot outline and rectangles border

        # make the visualization range map into the width and height of the world
        xrang = (-0.5, self.world.width+0.5) # was 0.5
        yrang = (-0.5, self.world.width+0.5) # was 0.5
        self.fig = figure(tools='', title="Environment + robots", plot_width=400, plot_height=400, 
                   x_range=xrang, y_range=yrang)
        self.fig.xgrid.grid_line_color = self.fig.ygrid.grid_line_color = None
        self.fig.axis.visible = False

        self.source_robots = ColumnDataSource({})
        self.source_env2 = ColumnDataSource({})
        self.source_im = ColumnDataSource({})

        self.update_env_robot_visualization(push=False)

        mapper = LinearColorMapper(high=1.0, low=0.0, palette="Viridis256")
        self.source_env2.data = {'x':[0], 'y':[0], 'dw':[10], 'dh':[10], 'image':[self.world.env.value] }
        self.fig.image(x = 'x', y = 'y', dw='dw', dh='dh', source=self.source_env2, image ='image', color_mapper=mapper)
        # robots are circles, their text is text, their history is a multi-line
        
        self.fig.circle(source = self.source_robots, x = 'robotx', y = 'roboty', radius=0.2, fill_color = 'blue')
        self.fig.text(source = self.source_robots, x = 'robotx', y = 'roboty', text='name', color = 'red')
        self.fig.multi_line(source = self.source_robots, xs = 'robotxs', ys = 'robotys', color = 'yellow')
        
        ## 
        
        self.fig_im = figure(tools='', title="Information model", plot_width=400, plot_height=400, 
                   x_range=xrang, y_range=yrang)
        self.source_im.data = {'x':[0], 'y':[0], 'dw':[10], 'dh':[10], 'image':[self.world.im.value] }
        
        self.fig_im.image(x = 'x', y = 'y', dw='dw', dh='dh', source=self.source_im, image ='image', 
                          color_mapper=mapper)
        
        show(row(self.fig, self.fig_im), notebook_handle=True)
        
        
    def create_visualization(self):
        self.create_env_robot_visualization()
        self.create_world_control_panel()
        self.update_visualization()
        
    def update_visualization(self):
        self.html_world_time.value = f"<b>T</b>={self.world.time}" 
        for r in self.world.robots:
            status = self.dict_robot_status[r.name]
            status.value = r.toHTML()
        self.update_env_robot_visualization()
        
    def update_env_robot_visualization(self, push = True):
        """Update the figure for the visualization of the visualization for the robot and the enviromment"""
        dict_robots = {}
        dict_robots["robotx"] = [r.x for r in self.world.robots]
        dict_robots["roboty"] = [r.y for r in self.world.robots]
        dict_robots["name"] = [r.name for r in self.world.robots]

        ### FIXME: trying out if I can plot the history like this
        dict_robots["robotxs"] = [[r.x, r.x-3, r.x-5] for r in self.world.robots]
        dict_robots["robotys"] = [[r.y, r.y, r.y+7] for r in self.world.robots]
        
        
        self.source_robots.data = dict_robots

        self.source_env2.data = {'x':[0], 'y':[0], 'dw':[10], 'dh':[10], 'image':[self.world.env.value] }
        self.source_im.data = {'x':[0], 'y':[0], 'dw':[10], 'dh':[10], 'image':[self.world.im.value]}
        #self.source_env = dict_env
        if push:
            push_notebook()
        

In [ ]:
# environment model
env = PollutionModelEnvironment("water", 10, 10)
env.evolve_speed = 1
env.p_pollution = 0.1
for i in range(100):
    env.proceed(1.0)
# setting it to unchanging
env.evolve_speed = 0
env.p_pollution = 0.0
plt.imshow(env.value)
# information model
#im = ScalarFieldInformationModel_stored_observation("sample", env.width, env.height, 
#                                estimation_type="disk-fixed", estimation_radius=2)
im = ScalarFieldInformationModel_stored_observation("sample", env.width, env.height, 
                                estimation_type="gaussian-process")
im.proceed(0)

In [ ]:
world = World(env, im)
robot = Robot("Robi", 2, 3, 0, env=env, im=im)
# robot.policy = GoToLocationPolicy(None, robot, 9, 9, 0.2)
world.add_robot(robot)
robot = Robot("Robur", 4, 8, 0, env=env, im=im)
robot.policy = RandomWaypointPolicy(None, robot, 1, [0,0], [9, 9])
world.add_robot(robot)
robot = Robot("R2D3", 5, 9, 0, env=env, im=im)
robot.policy = FollowPathPolicy(None, robot, 1, [[0,0], [5, 5], [9,0]], repeat = True)
world.add_robot(robot)

In [ ]:
ippui = IPPUI(world)
ippui.create_visualization()
#ippui.update_visualization()
#ippui.create_world_control_panel()

In [ ]:
plt.imshow(world.im.uncertainty)

In [ ]:
[r.name for r in world.robots]

In [ ]:
[r.x for r in world.robots]